# Over dit document
In dit document wordt een CRISP-DM cycles doorlopen, welke vervolgens een dataproduct (visualisaties) op zal leveren. Tot slot wordt er een conclusie getrokken en advies gegeven. Bij dit advies zal het ethische aspect meegenomen worden en dat zal de meeste nadruk krijgen.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_model import ARMA
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn import linear_model

df=pd.read_excel("Complete-dataset-FINAL.xlsx")


# Business understanding
Het ziekenhuis Isala wil de zorg voor diabetespatiënten verbeteren. Het gaat hier specifiek om het behandelproces door onnauwkeurigheden in metingen te filteren en hierop te baseren of een behandelplan aangepast moet worden of niet. Visualisaties zullen helpen bij het verkrijgen van inzicht van diverse meetmethoden om vervolgens conclusies te kunnen trekken. Die zijn er nog niet. Daarbij zou een visualisatie van een voorspelling meer inzicht brengen in hoe de total error mogelijk zal veranderen per jaar. Dit zou kunnen resulteren in toekomstige plannen voor het standaardiseren van de HbA1c meetmethoden.

Om een voorspelling te realiseren & visualiseren wordt het ARIMA model gebruikt. Als eerste wordt er gekeken naar hoe stationair de data is. Vervolgens zullen de parameters (zie uitleg ARIMA model) voor dit model worden afgestemd op verschillende grafieken, ARIMA(p=?, d=?,q=?). Tot slot zal de voorspelling gevisualiseerd worden.

#### Uitleg ARIMA model
Het doel van een ARIMA model is het nabootsen van een tijdsserie. Dit wordt gedaan door de variaties in de data te modelleren door middel van de volgende drie opties: 

- AR (auto-regressief), voorgaande waardes worden gebruikt om nieuwe waardes te voorspellen.
- I (integrated), niet de originele serie maar een gedifferentieerde tijdsserie wordt gebruikt. Dit om de tijdsserie stationair te maken. 
- MA (moving average), voorafgaande fouten worden gebruikt om nieuwe fouten voorspellen. Dit heeft een smoothing effect, een bewegend gemiddelde. 

Door deze effecten te mengen kun je de meeste tijdsseries nabootsen. In de volgende sectie gaan we de effecten van de AR en MA termen op een tijdsserie bekijken. 

Note

In dit notebook zal er gefocust worden op het ARIMA model. Het aantal metingen per fabrikant verschilt enorm. Daarom wordt er nu gefocust op de top 10 fabrikanten met de meeste metingen.

# Data understanding
Welke databronnen zijn gegeven en in welk formaat:
Een excel bestand genaamd: Complete-dataset-FINAL.xlsx
##### note
Het excel bestand dat gebruikt gaat worden, is het resultaat van rapporten die om zijn gezet naar excel. Dit is de meest recente versie waarin de meetmethoden zijn genormaliseerd en de waarden dubbelgecheckt zijn.

Hoe groot zijn deze databronnen:
355 kB

-4685 rijen

-12 kolommen



Kolomnamen: Method name, N (no. labs), Mean, Bias, CV, Sample, Reference value, Year, Source, Type, Manufacturer (fabrikant).
- Method name : Naam van het meetmethode
- N	: Aantal deelgenomen labs
- Mean :  Gemiddelde van alle HbA1c waarde
- Bias : Absolute toe- of afname van de hoeveelheid mmol/mol ten opzichte van de werkelijke waarde
- CV : Laat zien wat de spreiding is de metingen
- Sample : Hoeveelste meting in het jaar
- Reference Value : Variabel die is gebaseerd op 95% van de gezonde populatie		
- Year : Welk jaar de meting is gedaan
- Source	: Amerikaanse of Europese data
- Type : Vers of bevroren samples

Datatypes: Object, float en int (jaar) 

# Data preparation
De dataset wordt aangepast, zodat het bruikbaar is om tijdsreeksen mee te kunnen voorspellen.

In [ ]:
#Check to drop all empty values, adjust column name so it's usable
df=df.dropna()
df.columns = df.columns.str.replace('Total Error', 'Total')
df.columns

In [ ]:
df.Year.astype('int32')
df['Year'] =pd.to_datetime(df.Year, format='%Y')
df.info()

In [ ]:
# df = df.set_index('Year')
df.head()

In [ ]:
# predict only top 10 manufacturers with most datapoints
list_top10 = df['Manufacturer'].value_counts()[:10].index.tolist()
top10_manufacturers = df.loc[df['Manufacturer'].isin(list_top10)]
top10_manufacturers['Manufacturer'].value_counts()
top10_manufacturers.info()

In [ ]:
top10_manufacturers=top10_manufacturers[['Manufacturer', 'Total']]
manufacturers = top10_manufacturers.groupby("Manufacturer")
manufacturers.size().nlargest(20)

In [ ]:
#quick overview of the data
for name, data in manufacturers:
    print(name)
    plt.plot(data.index, data['Total'])
    plt.show()

# Note
Hier is te zien dat er niet een duidelijke lijn wordt weergegeven van door de jaren heen. Dit is een indicatie dat in de dataset de datapunten nog moeten worden gesorteerd. Bij de volgende grafieken zijn de datapunten gesorteerd.

In [ ]:
manufacturers_top10 = top10_manufacturers.sort_values(by="Year")
manufacturers_top10 = manufacturers_top10.reset_index()

manufacturers_top10=manufacturers_top10[['Manufacturer', 'Total']]
manufacturers_sorted = manufacturers_top10.groupby("Manufacturer")

for name, data in manufacturers_sorted:
    print(name)
    plt.plot(data.index, data['Total'])
    plt.show()

# Analyse lijnplot
In de gesorteerde grafieken is redelijk een stationaire lijn te zien. Dit zal worden meegenomen in het bepalen van de parameters van het ARIMA model.

# Modeling : parameters ARIMA

In [ ]:
# Check for stationarity of the time-series data
# We will look for p-value. In case, p-value is less than 0.05, the time series data can said to have stationarity.
from statsmodels.tsa.stattools import adfuller
List_meetmethoden = []


#AIC input is to compute the optimal number iteratively.
for name, data in manufacturers: 
    data= data.dropna()
    print(name) 
    df_stationarityTest = adfuller(data['Total'], autolag='AIC')    
    
    # data['Total'].plot(figsize=(16,10))
    # plt.show() 
    print("\n")
    print(f'ADF Statistic: {df_stationarityTest[0]}')
    print(f'n_lags: {df_stationarityTest[1]}')
    print(f'p-value: {df_stationarityTest[1]}')
    for key, value in df_stationarityTest[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')    

    
    if df_stationarityTest[1] > 0.05:
        List_meetmethoden.append(name)
    print("\n")

print("Meetmethoden met een p-waarden boven de 0.05:")
List_meetmethoden

### Analyse van p-values
Bij een p-value van 0.05 of er onder, betekent het dat je met zekerheid kan zeggen dat er een relatie is. Wanneer de p-value hoger is dan 0.05 betekent het dat er een kans is op een nulhypothese (voorspelling met geen effect of enige relatie).
Om deze meetmethoden onder de 0.05 te krijgen, wordt het gedifferentieerd. 

#### Differentiatie van meetmethoden met p-value > 0.05

In [ ]:
#Één keer diffrentiëren
for name, data in manufacturers:
    if name in List_meetmethoden:
        print("\n" + name) 
        df_stationarityTest = adfuller(data['Total'].diff().dropna(), autolag='AIC')   
        # data['Total'].diff().plot(figsize=(16,10))
        plt.show() 
        
        print("")
        print(f'ADF Statistic: {df_stationarityTest[0]}')
        print(f'n_lags: {df_stationarityTest[1]}')
        print(f'p-value: {df_stationarityTest[1]}')
        for key, value in df_stationarityTest[4].items():
            print('Critial Values:')
            print(f'   {key}, {value}')    

        

In [ ]:
#2x diff = I (ARIMA)
for name, data in manufacturers:
    if name in List_meetmethoden:
        print("\n")
        print(name) 
        # data['Total'].diff().diff().plot(figsize=(16,10))
        # plt.show()
        data_diff = data['Total'].diff().diff().dropna()
        
        df_stationarityTest = adfuller(data_diff, autolag='AIC')    

        print(f'ADF Statistic: {df_stationarityTest[0]}')
        print(f'n_lags: {df_stationarityTest[1]}')
        print(f'p-value: {df_stationarityTest[1]}')
        for key, value in df_stationarityTest[4].items():
            print('Critial Values:')
            print(f'   {key}, {value}')    



# Conclusie p-waarden
Hier is te zien dat bij twee keer differentiëren dat de p-waarden juist groter worden. Voor het ARIMA model nemen we de kleinste p-waarde. Daarom nemen we 0 als parameter bij alle meetmethoden. Dus ARIMA(p=?,d=0,q=?)


# ACF & PACF

### ACF plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
for name, data in manufacturers: 
    acf =plot_acf(data['Total'], title=name, alpha=.05)
    

### Analyse van ACF plot
In dit autocorrelatieplot liggen bijna alle waardes buiten het lichtblauwe onzekerheidsgebied (deze variantie in autocorrelatie kan mogelijk ook worden verklaard door ruis). Daarnaast zijn de meeste punten positief. Dit zou betekenen dat we juist wel moeten differentiëren.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
for name, data in manufacturers: 
    acf =plot_acf(data['Total'].diff().dropna(), title=name, alpha=.05)

# Analayse acf plot met differentiëren
Na het differentiëren ziet de acf plot er gelijk beter uit. Zo is er een verdeling tussen positieve en negatieve waarden en vallen de meeste datapunten binnen het onzekerheidsgebied. In kort laat de Adfuller test en de acf plot verschillende uitgangspunten zien. 

Achteraf is hier feedback over gevraagd aan Ruben. Hieruit is gekomen dat de Adfullter test resultaat als uitgangspunt zal worden genomen.
D is 0, dus ARIMA(p=?,d=0,q=?)
Voor het bepalen van de q parameter kijken we naar de datapunten die buiten de onzekerheidgebied vallen in de acf plot.

In [ ]:
#This list is created by just counting the data points that are outside the confidence interval
Lijst_MA = [3,2,1,6,4,1,2,2,5,1]

# Pacf plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

for name, data in manufacturers: 
    pacf = plot_pacf(data['Total'], title=name)

### Analyse van pacf plot
De partiële autocorrelatie geeft weer hoe sterk het verband is tussen de waarde van een lag en de waarde van voorgaande lags. Als er tussen de lags nog significante correlatie bestaat is dit een aanwijzing dat er auto-correlatie optreed en dat het instellen van de AR parameter een goed idee is.

Bij alle grafieken is te zien dat de eerste lag het meest significant is. Daarom nemen we p met de waarde 1. Dus ARIMA(p=1,d=0,q=lijst_MA)

# ARIMA modeling

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import datetime

## ARIMA summary output

In [ ]:
counter= 0

for name, data in manufacturers:
    print(name)
    data.index = pd.DatetimeIndex(data.index).to_period('M')

    model = sm.tsa.arima.ARIMA(data['Total'].diff().dropna(), order=(1,0,Lijst_MA[counter]))
    model_fit = model.fit()
    print(model_fit.summary())
    counter = counter +1
    
   
#P>|z| significant? onder 0.01

## ARIMA prediction visualisation

In [ ]:
counter= 0
from sklearn.model_selection import train_test_split

import datetime
for name, data in manufacturers:
    tempdf = data[['Total']].dropna()
    X = tempdf.iloc[:, :1].values
    Y = tempdf.index.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

    model = sm.tsa.arima.ARIMA(X_train, order=(1,0,Lijst_MA[counter]))
    model_fit = model.fit()
    counter = counter +1 #next in list

    y_pred = model_fit.predict(disp=0, exog=None, dynamic=False)
    length_predicted_values = len(y_pred)


    preddf = pd.DataFrame(y_pred, columns=['Total'])
    preddf['Year'] = pd.to_datetime('2022-01-01')
    preddf = preddf.set_index('Year')
    
    
    # Add tempdf and preddf together and reset index
    tempdf = pd.concat([tempdf, preddf], axis=0)

    tempdf.reset_index(inplace=True)
    
    #find predicted value in concatenated dataframe
    start_index_predValues = max(tempdf.index)-length_predicted_values    
    real_values_df = tempdf.loc[:start_index_predValues]
   
    #sort by year and reset index so all years line up
    real_values_df = real_values_df.sort_values(by="Year")
    real_values_df = real_values_df.reset_index()
    
    l = sns.lineplot(x=real_values_df.index, y='Total', data=real_values_df, hue=real_values_df['Year'])
    l.set_title('Total error prediction: ' + name )
    
    #show predicted values
    predict_values_df = tempdf.loc[start_index_predValues:]
    sns.lineplot(x=predict_values_df.index, y='Total', data=predict_values_df, color='orange')
    sns.set(rc = {'figure.figsize':(25,8)})
    plt.show()

# Evaluation


Tijdens het bepalen van de arima modellen zijn bij 2 meetmethoden geen geschikte p-waarde gevonden. Namelijk bij de 'Abott' en 'Roche diagonistics'. Doordat de p-waarde niet onder de 0.05 is, moet er bij de voorspelling van deze meetmethoden rekening worden gehouden met een groter onzekerheidsgebied.


#### Conclusie
Er zijn bij alle de meetmethoden op te merken dat in het jaar 2022 (oranje-lijntje) een sterke daling of stijging wordt voorspeld, waarbij diverse meetmethoden nog wat pieken gaan krijgen en dan in de meeste gevallen zullen stabiliseren. 


#### Advies
Het advies voor de opdrachtgever is derhalve om dit model en deze visualisaties slechts als indicatie te gebruiken. Dit wordt aanbevolen op basis van de ethische vraag 'Kunnen deze visualisaties leiden tot gepaste, ethische conclusies?'.

Als het gaat om de waarde 'precisie', zullen deze visualisaties niet leiden tot gepaste ethische conclusies. De metingen die in de rapporten staan, zijn genoteerd zonder accurate datums. Hierdoor is de voorspelling gemaakt op data waarvan de datums niet bekend zijn, terwijl deze datums bij het gebruikte model een essentieel component zijn. Door dat deze datums niet zijn meegenomen, is de voorspelling derhalve minder nauwkeurig. Daarom is het advies om deze visualisaties slechts als indicatie te gebruiken en niet als richtlijn. 

    
